# Accuracy analysis

In [ ]:
import json
import os
import pathlib

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import pandas as pd
import pyrootutils
import regex as re
import seaborn as sns
import sklearn.metrics as sk_metrics

In [ ]:
PROJECT_ROOT = pyrootutils.find_root(
    search_from=os.path.abspath(""), indicator=".project-root"
)

## Load data & model responses

In [ ]:
YES_RE = re.compile(r"[^a-zA-Z]*\b(yes|no)\b[^a-zA-Z]*", re.IGNORECASE)


def extract_content(choices_list: list) -> str:
    return choices_list[0]["message"]["content"]


def extract_prediction(response: str) -> str:
    # get a list of all matches to YES_RE in `response`; take the last match
    # and check if it is a "yes" or "no" response

    matches = YES_RE.findall(response)
    if len(matches) == 0:
        return "unknown"
    else:
        last_match = matches[-1]
        if last_match.lower() == "yes":
            return "positive"
        else:
            return "negative"

In [ ]:
completions_path = PROJECT_ROOT / "data" / "completions"

# list all files matching ".*_inputs.jsonl" in the completions_path
inputs_files = list(completions_path.glob("*_inputs.jsonl"))
results_files = list(completions_path.glob("*_results.jsonl"))

# Grab the batch_id from the filename
batch_id_re = re.compile(r"^(batch_\w+)_")

inputs_dfs = []
for f in inputs_files:
    i_df = pd.read_json(f, lines=True)
    i_json_struct = json.loads(i_df.to_json(orient="records"))
    i_flat_df = pd.json_normalize(i_json_struct)
    batch_id = batch_id_re.search(f.name).group(1)
    i_flat_df["batch_id"] = batch_id
    inputs_dfs.append(i_flat_df)
inputs_df = pd.concat(inputs_dfs, ignore_index=True)

results_dfs = []
for f in results_files:
    r_df = pd.read_json(f, lines=True)
    r_json_struct = json.loads(r_df.to_json(orient="records"))
    r_flat_df = pd.json_normalize(r_json_struct)
    batch_id = batch_id_re.search(f.name).group(1)
    r_flat_df["batch_id"] = batch_id
    results_dfs.append(r_flat_df)
results_df = pd.concat(results_dfs, ignore_index=True)

# Merge inputs and results on the the batch_id and custom_id
response_full_df = results_df.merge(
    inputs_df[
        [
            "custom_id",
            "batch_id",
            "body.metadata.sample_type",  # ground-truth label for sample
            "body.metadata.sample",  # the sample itself
            "body.metadata.grammar_file",  # grammar file used
            "body.metadata.model",  # model used
            "body.metadata.n_shots",  # n_shots used
        ]
    ],
    on=["batch_id", "custom_id"],
)

response_full_df = response_full_df.rename(
    columns={
        "body.metadata.sample_type": "sample.type.ground_truth",
        "body.metadata.sample": "sample",
        "body.metadata.grammar_file": "grammar_file",
        "body.metadata.model": "model",
        "body.metadata.n_shots": "n_shots",
    }
)

response_full_df["model_response"] = response_full_df["response.body.choices"].apply(
    extract_content
)

response_df = response_full_df[
    [
        "sample",
        "sample.type.ground_truth",
        "model_response",
        "grammar_file",
        "model",
        "n_shots",
    ]
].copy()


response_df["sample.type.predicted"] = response_df["model_response"].apply(
    extract_prediction
)

response_df["sample.length"] = response_df["sample"].apply(
    lambda s: len(str(s).split(" "))
)

response_df["correct"] = (
    response_df["sample.type.ground_truth"] == response_df["sample.type.predicted"]
)

response_df = response_df.dropna()

response_df["n_shots"] = pd.Categorical(
    response_df["n_shots"],
    categories=["0", "2", "4", "8", "16", "32"],
    ordered=True,
)
response_df["sample.type.ground_truth"] = pd.Categorical(
    response_df["sample.type.ground_truth"],
    categories=["positive", "negative"],
    ordered=True,
)
response_df["sample.type.predicted"] = pd.Categorical(
    response_df["sample.type.predicted"],
    categories=["positive", "negative", "unknown"],
    ordered=True,
)
response_df["model"] = pd.Categorical(
    response_df["model"],
)

unique_grammars = response_df["grammar_file"].unique()
keep_grammars = [
    "sample_trim_20241022141559",
    "sample_trim_20250115102355",
    "sample_trim_20250115225054",
    "sample_trim_20250116110034",
    "sample_trim_20250206142600",
    "sample_trim_20250206142636",
    "sample_trim_20250206142703",
    "sample_trim_20250206142726",
    "sample_trim_20250206142746",
    "sample_trim_20250206142808",
    "sample_trim_20250206142843",
    "sample_trim_20250206142919",
    "sample_trim_20250206142942",
    "sample_trim_20250206143028",
    "sample_trim_20250206143054",
    "sample_trim_20250206143126",
    "sample_trim_20250206143209",
    "sample_trim_20250207140159",
]
response_df = response_df[response_df["grammar_file"].isin(keep_grammars)]

g_map_dict = {g: f"Grammar {i+1}" for i, g in enumerate(keep_grammars)}
response_df["grammar_name"] = response_df["grammar_file"].map(g_map_dict)
response_df["grammar_name"] = pd.Categorical(
    response_df["grammar_name"],
    categories=list(g_map_dict.values()),
    ordered=True,
)

In [ ]:
response_df.info()

In [ ]:
g_map_dict

In [ ]:
(
    response_df.groupby(
        ["grammar_name", "n_shots", "sample.type.ground_truth", "model"]
    )[["sample"]].count()
)

## Plot sample-length distribution

In [ ]:
fig = plt.figure(figsize=(6, 3))

ax = fig.add_subplot(111)

sns.histplot(
    data=response_df,
    x="sample.length",
    ax=ax,
    binwidth=1,
    hue="sample.type.ground_truth",
    palette={"positive": "orange", "negative": "purple"},
)

ax.get_legend().set_title("Sample type")

ax.set_yscale("log")
ax.set_xlabel("Sample length")

Since some of longer sample lengths only have a few samples, the variance on the 
accuracy will be really high. We solve this by throwing out any samples without at least
10 samples in that length category.

In [ ]:
MIN_NUM_SAMPLES = 20

samples_by_length = response_df.groupby("sample.length")["sample"].count()
many_samples_lengths = samples_by_length[
    samples_by_length > MIN_NUM_SAMPLES
].index.values

response_df = response_df[response_df["sample.length"].isin(many_samples_lengths)]

## Calculate accuracy metrics

In [ ]:
mean_accuracy = sk_metrics.accuracy_score(
    response_df["sample.type.ground_truth"], response_df["sample.type.predicted"]
)

mean_cm = sk_metrics.confusion_matrix(
    response_df["sample.type.ground_truth"],
    response_df["sample.type.predicted"],
    normalize="true",
)

negative_sample_acc = mean_cm[0][0]
positive_sample_acc = mean_cm[1][1]

In [ ]:
ax = plt.subplot()

sns.heatmap(data=mean_cm, annot=True, ax=ax, vmin=0.0, vmax=1.0, cmap="coolwarm")

ax.set_xlabel("Predicted Label")
ax.set_xticklabels(["Negative", "Positive", "Unknown"])
ax.set_ylabel("True Label")
ax.set_yticklabels(["Negative", "Positive", "Unknown"])

## Plot accuracy by sample length & type

In [ ]:
fig = plt.figure(figsize=(6, 5))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex=ax0)

n_bins = response_df["sample.length"].nunique()

sns.histplot(
    data=response_df,
    x="sample.length",
    ax=ax0,
    binwidth=1,
    color="gray",
)

sns.lineplot(
    data=response_df,
    x="sample.length",
    y="correct",
    hue="sample.type.ground_truth",
    ax=ax1,
    style="model",
    palette={"positive": "orange", "negative": "purple"},
    # markers=["o", "o"],
    markers=True,
    # dashes=False,
    alpha=0.5,
    linewidth=2,
    err_style="bars",
)

ax0.set_yscale("log")
ax0.set_ylim(10, None)

ax1.set_ylabel("Mean accuracy")
ax1.set_xlabel("Sample length")

ax1.get_legend().set_title("Sample type")

# hide x-axis label and tick labels on the first subplot
ax0.set_xlabel("")
ax0.tick_params(axis="x", which="both", bottom=True, top=False, labelbottom=False)

## Few-shot analysis

In [ ]:
fig = plt.figure(figsize=(13, 3))
gs = gridspec.GridSpec(1, 4)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3]

for i in range(len(axes)):
    sns.lineplot(
        data=response_df[response_df["grammar_name"] == f"Grammar {i+1}"],
        x="n_shots",
        y="correct",
        hue="sample.type.ground_truth",
        ax=axes[i],
        style="model",
        palette={"positive": "orange", "negative": "purple"},
        markers=True,
        alpha=0.5,
        linewidth=2,
        markersize=8,
        err_style="bars",
    )

    sns.lineplot(
        data=response_df[response_df["grammar_name"] == f"Grammar {i+1}"],
        x="n_shots",
        y="correct",
        ax=axes[i],
        color="black",
        style="model",
        markers=True,
        errorbar=None,
        linewidth=3,
        markersize=8,
        legend=False,
    )

handles, labels = axes[-1].get_legend_handles_labels()
labels[0] = "Sample Type"
labels[3] = ""

axes[-1].legend(
    loc="upper left",
    bbox_to_anchor=(1, 1),
    handles=handles,
    labels=labels,
)

_ = axes[0].set_ylim(-0.02, None)
_ = axes[0].set_ylabel("Mean Accuracy (pass@1)")
_ = axes[0].set_xlabel("# of Shots [log scale]")
_ = axes[0].set_title("Grammar 1")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Shots [log scale]")
    _ = axes[i].set_title(f"Grammar {i+1}")

In [ ]:
fig = plt.figure(figsize=(13, 3))
gs = gridspec.GridSpec(1, 4)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3]

f1_df = (
    response_df.groupby(["n_shots", "model", "grammar_name"])
    .apply(
        lambda group: sk_metrics.f1_score(
            group["sample.type.ground_truth"],
            group["sample.type.predicted"],
            average="weighted",
        )
    )
    .reset_index(name="f1_score")
)

for i in range(len(axes)):
    sns.lineplot(
        data=f1_df[f1_df["grammar_name"] == f"Grammar {i+1}"],
        x="n_shots",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlabel("# of Shots [log scale]")
_ = axes[0].set_title("Grammar 1")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Shots [log scale]")
    _ = axes[i].set_title(f"Grammar {i+1}")

In [ ]:
zero_shots_df = response_df[response_df["n_shots"] == "0"]
two_shots_df = response_df[response_df["n_shots"] == "2"]
four_shots_df = response_df[response_df["n_shots"] == "4"]
eight_shots_df = response_df[response_df["n_shots"] == "8"]

mean_cm_0 = sk_metrics.confusion_matrix(
    zero_shots_df["sample.type.ground_truth"],
    zero_shots_df["sample.type.predicted"],
    normalize="true",
)

mean_cm_2 = sk_metrics.confusion_matrix(
    two_shots_df["sample.type.ground_truth"],
    two_shots_df["sample.type.predicted"],
    normalize="true",
)

mean_cm_4 = sk_metrics.confusion_matrix(
    four_shots_df["sample.type.ground_truth"],
    four_shots_df["sample.type.predicted"],
    normalize="true",
)

mean_cm_8 = sk_metrics.confusion_matrix(
    eight_shots_df["sample.type.ground_truth"],
    eight_shots_df["sample.type.predicted"],
    normalize="true",
)

fig = plt.figure(figsize=(13, 3))
gs = gridspec.GridSpec(1, 4)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0)

sns.heatmap(
    data=mean_cm_0, annot=True, ax=ax0, vmin=0.0, vmax=1.0, cmap="coolwarm", cbar=False
)
sns.heatmap(
    data=mean_cm_2, annot=True, ax=ax1, vmin=0.0, vmax=1.0, cmap="coolwarm", cbar=False
)
sns.heatmap(
    data=mean_cm_4, annot=True, ax=ax2, vmin=0.0, vmax=1.0, cmap="coolwarm", cbar=False
)
sns.heatmap(
    data=mean_cm_8, annot=True, ax=ax3, vmin=0.0, vmax=1.0, cmap="coolwarm", cbar=False
)


ax0.set_ylabel("True Label")
ax0.set_yticklabels(["Negative", "Positive", "Unknown"])

ax0.set_xlabel("Predicted Label")
ax0.set_xticklabels(["Negative", "Positive", "Unknown"])
ax0.set_title("0 Shot")
ax1.set_xlabel("Predicted Label")
ax1.set_xticklabels(["Negative", "Positive", "Unknown"])
ax1.set_title("2 Shot")
ax2.set_xlabel("Predicted Label")
ax2.set_xticklabels(["Negative", "Positive", "Unknown"])
ax2.set_title("4 Shot")
ax3.set_xlabel("Predicted Label")
ax3.set_xticklabels(["Negative", "Positive", "Unknown"])
ax3.set_title("8 Shot")

ax1.tick_params(axis="y", which="both", left=True, right=False, labelleft=False)
ax2.tick_params(axis="y", which="both", left=True, right=False, labelleft=False)
ax3.tick_params(axis="y", which="both", left=True, right=False, labelleft=False)

## Grammar Complexity Analysis

In [ ]:
complexity_stats = pd.DataFrame(
    [
        {
            "grammar_file": "sample_trim_20241022141559",
            "n_terminals": 4,
            "n_nonterminals": 6,
            "n_lexical_productions": 8,
            "n_nonlexical_productions": 7,
            "compression_ratio": 9.643571,
        },
        {
            "grammar_file": "sample_trim_20250115102355",
            "n_terminals": 9,
            "n_nonterminals": 11,
            "n_lexical_productions": 10,
            "n_nonlexical_productions": 20,
            "compression_ratio": 6.756489,
        },
        {
            "grammar_file": "sample_trim_20250115225054",
            "n_terminals": 19,
            "n_nonterminals": 19,
            "n_lexical_productions": 27,
            "n_nonlexical_productions": 19,
            "compression_ratio": 6.108935,
        },
        {
            "grammar_file": "sample_trim_20250116110034",
            "n_terminals": 38,
            "n_nonterminals": 47,
            "n_lexical_productions": 78,
            "n_nonlexical_productions": 64,
            "compression_ratio": 4.443383,
        },
    ]
)

complexity_stats["grammar_name"] = complexity_stats["grammar_file"].map(g_map_dict)

complexity_stats

In [ ]:
from formal_gym import grammar as fg_grammar

g = fg_grammar.Grammar.from_grammar(
    PROJECT_ROOT / "data" / "grammars" / "sample_trim_20250116110034.cfg"
)
print(g.n_terminals)
print(g.n_nonterminals)
print(g.n_lexical_productions)
print(g.n_non_lexical_productions)

In [ ]:
fig = plt.figure(figsize=(16, 3))
gs = gridspec.GridSpec(1, 6)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)
ax4 = plt.subplot(gs[4], sharey=ax0, sharex=ax0)
ax5 = plt.subplot(gs[5], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3, ax4, ax5]
shot_lens = [0, 2, 4, 8, 16, 32]

f1_df = (
    response_df.groupby(["n_shots", "model", "grammar_name"])
    .apply(
        lambda group: sk_metrics.f1_score(
            group["sample.type.ground_truth"],
            group["sample.type.predicted"],
            average="weighted",
        )
    )
    .reset_index(name="f1_score")
)

merged_df = pd.merge(f1_df, complexity_stats, on="grammar_name")

for i in range(len(axes)):
    sns.lineplot(
        data=merged_df[merged_df["n_shots"] == f"{shot_lens[i]}"],
        x="n_terminals",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlabel("# of Terminals")
_ = axes[0].set_title("0 Shots")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Terminals")
    _ = axes[i].set_title(f"{shot_lens[i]} Shots")

In [ ]:
fig = plt.figure(figsize=(16, 3))
gs = gridspec.GridSpec(1, 6)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)
ax4 = plt.subplot(gs[4], sharey=ax0, sharex=ax0)
ax5 = plt.subplot(gs[5], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3, ax4, ax5]
shot_lens = [0, 2, 4, 8, 16, 32]

for i in range(len(axes)):
    sns.lineplot(
        data=merged_df[merged_df["n_shots"] == f"{shot_lens[i]}"],
        x="n_nonterminals",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlabel("# of Nonterminals")
_ = axes[0].set_title("0 Shots")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Nonterminals")
    _ = axes[i].set_title(f"{shot_lens[i]} Shots")

In [ ]:
fig = plt.figure(figsize=(16, 3))
gs = gridspec.GridSpec(1, 6)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)
ax4 = plt.subplot(gs[4], sharey=ax0, sharex=ax0)
ax5 = plt.subplot(gs[5], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3, ax4, ax5]
shot_lens = [0, 2, 4, 8, 16, 32]

for i in range(len(axes)):
    sns.lineplot(
        data=merged_df[merged_df["n_shots"] == f"{shot_lens[i]}"],
        x="n_lexical_productions",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlabel("# of Lexical Rules")
_ = axes[0].set_title("0 Shots")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Lexical Rules")
    _ = axes[i].set_title(f"{shot_lens[i]} Shots")

In [ ]:
fig = plt.figure(figsize=(16, 3))
gs = gridspec.GridSpec(1, 6)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)
ax4 = plt.subplot(gs[4], sharey=ax0, sharex=ax0)
ax5 = plt.subplot(gs[5], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3, ax4, ax5]
shot_lens = [0, 2, 4, 8, 16, 32]

for i in range(len(axes)):
    sns.lineplot(
        data=merged_df[merged_df["n_shots"] == f"{shot_lens[i]}"],
        x="n_nonlexical_productions",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlabel("# of Nonlexical Rules")
_ = axes[0].set_title("0 Shots")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("# of Nonlexical Rules")
    _ = axes[i].set_title(f"{shot_lens[i]} Shots")

In [ ]:
fig = plt.figure(figsize=(16, 3))
gs = gridspec.GridSpec(1, 6)

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharey=ax0, sharex=ax0)
ax2 = plt.subplot(gs[2], sharey=ax0, sharex=ax0)
ax3 = plt.subplot(gs[3], sharey=ax0, sharex=ax0)
ax4 = plt.subplot(gs[4], sharey=ax0, sharex=ax0)
ax5 = plt.subplot(gs[5], sharey=ax0, sharex=ax0)

axes = [ax0, ax1, ax2, ax3, ax4, ax5]
shot_lens = [0, 2, 4, 8, 16, 32]

for i in range(len(axes)):
    sns.lineplot(
        data=merged_df[merged_df["n_shots"] == f"{shot_lens[i]}"],
        x="compression_ratio",
        y="f1_score",
        hue="model",
        style="model",
        markers=True,
        ax=axes[i],
        linewidth=2,
        markersize=8,
    )

# draw a dashed vertical line at x = 1
ax0.axvline(x=1, color="red", linestyle=":")

_ = axes[0].set_ylim(-0.02, 1.02)
_ = axes[0].set_xlim(0.8, None)
_ = axes[0].set_xlabel("Compression Ratio")
_ = axes[0].set_title("0 Shots")
_ = axes[0].set_ylabel("F1 Score")

for ax in axes[:-1]:
    _ = ax.get_legend().remove()

for i in range(1, len(axes)):
    _ = axes[i].set_ylabel(None)
    _ = axes[i].set_xlabel("Compression Ratio")
    _ = axes[i].set_title(f"{shot_lens[i]} Shots")
    _ = axes[i].axvline(x=1, color="red", linestyle=":")

In [ ]:
merged_df

## Positive sample proportions

In [ ]:
partitions_path = PROJECT_ROOT / "data" / "partitions"

# open all .csv files in partitions_path
partitions_files = list(partitions_path.glob("*.csv"))

# read all .csv files into a single dataframe
partitions_dfs = []
for f in partitions_files:
    f_name = pathlib.Path(f).stem.split("_k=")[0].split("counts_")[1]
    g_name = g_map_dict[f_name]
    p_df = pd.read_csv(f)
    p_df["grammar_file"] = g_name
    partitions_dfs.append(p_df)

partitions_df = (
    pd.concat(partitions_dfs, ignore_index=True)
    .groupby(["grammar_file", "sample.length"])
    .first()
    .reset_index()
)

In [ ]:
fig = plt.figure(figsize=(6, 4))
gs = gridspec.GridSpec(1, 1)
ax0 = plt.subplot(gs[0])

sns.lineplot(
    data=partitions_df,
    x="sample.length",
    y="prop_positive_samples",
    hue="grammar_file",
    style="grammar_file",
    linewidth=2,
    markers=True,
    # color="orange",
    palette="Oranges",
    ax=ax0,
)

ax0.set_ylabel("Proportion of Strings in Grammar")
ax0.set_ylim(0, 1)
ax0.get_legend().set_title("")
ax0.set_xlabel("Sample Length")